In [114]:
# Chargement des librairies
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
from statsmodels.regression.linear_model import OLS
%matplotlib inline

Nous souhaitons évaluer l’impact de différents facteurs dans le prix des voitures. Dans ce cadre on souhaite mettre en oeuvre différentes techniques d’analyse de données et de machine learning.

# Régression du prix des voitures

Chargement du fichier CSV

In [115]:
data_reg = pd.read_csv('cars_data-reg.csv')

On commence par regarder s'il y a un lien entre la valeur totale d'une voiture et les différents facteurs.

In [116]:
data_reg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Kilométrage        1000 non-null   int64  
 1   Année_Fabrication  1000 non-null   int64  
 2   Puissance          1000 non-null   int64  
 3   Consommation       1000 non-null   float64
 4   Nombre_Portes      1000 non-null   int64  
 5   Vitesse_Max        1000 non-null   int64  
 6   Marque             1000 non-null   int64  
 7   Prix               1000 non-null   float64
dtypes: float64(2), int64(6)
memory usage: 62.6 KB


On affiche les valeurs des différentes statistiques descriptives avec la fonction describe. 

In [117]:
data_reg.describe(percentiles=None, include=None, exclude=None)

,Kilométrage,Année_Fabrication,Puissance,Consommation,Nombre_Portes,Vitesse_Max,Marque,Prix
count,1000.0000,1000.00000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,146302.3380,2011.32600,235.010000,53.614884,3.514000,225.039000,2.000000,32355.779868
std,86102.8942,6.87772,64.058218,27.336137,1.159806,43.022953,0.811578,10368.675552
min,728.0000,2000.00000,125.000000,5.353732,2.000000,150.000000,1.000000,5666.794333
25%,70326.0000,2005.00000,181.000000,29.955260,2.000000,188.000000,1.000000,24671.936893
50%,143958.5000,2011.00000,237.000000,53.356480,4.000000,226.000000,2.000000,32058.370765
75%,223790.2500,2017.00000,288.000000,77.950275,5.000000,262.000000,3.000000,39789.299440
max,299790.0000,2023.00000,349.000000,99.979611,5.000000,300.000000,3.000000,59640.769068


On va faire les différentes régressions linéaires simples possibles, aifn de choisir les facteurs les plus pertinents. 

Des graphiques seront télécharger dans l'espace de lancement du script.


In [118]:
y =data_reg['Prix']
Colonnes = ['Kilométrage', 'Année_Fabrication','Puissance','Consommation','Nombre_Portes','Vitesse_Max','Marque']

c=1
for i in Colonnes:
    print('________________________________________________________________________________________________')
    print('Modèle n° ',c, "permettant de faire une régréssion simple entre le Prix et le/la/l'",i)
    c += 1
    
    X = data_reg[i]
    
    a,b,rvalue,p_value,_=stats.linregress(X,y)

    y_hat= a * X + b
    print('y_hat = ',y_hat)

    # R² = rvalue ** 2 
    # = (SCE/SCT)
    # R² correspond au coefficient de détermination; il mesure la variabilité expliquée par le modèle de régréssion linéaire. 
    # Plus sa valeur se rapproche de 0 plus le modèle est mauvais, plus il se rapproche de 1, plus il sera jugé excellent. 
    R2= rvalue ** 2
    
    print('    R^2 = ',R2)
    print('        ==> {:2.2%} de la variabilité du prix est expliquée par {}'.format(R2,i))
    print('    On teste l''hypothèse HO : beta_1 = 0 au risque alpha = 5%')
    print('    p_value : ',p_value)

    if p_value < .05 :
        print('On rejette H0 au risque 5%.')
        print(i, ' a bien un impact sur le prix')
    else:
        print('On accepte H0')

    # Visualisation
    plt.grid()
    plt.scatter(X, data_reg['Prix'], color='red', marker='x')
    plt.plot(X, y_hat, color='green')
    plt.xlabel(i)
    plt.ylabel('Prix')
    # Enregistrement du graphique dans un fichier image
    plt.savefig(f'{i}_graph.png')
    # On supprime le graphique de la mémoire afin d'éviter les superposition dans le prochain tour de boucle
    plt.clf()
    plt.close()

________________________________________________________________________________________________
Modèle n°  1 permettant de faire une régréssion simple entre le Prix et le/la/l' Kilométrage
y_hat =  0      26769.285257
1      27587.701600
2      22920.593616
3      32127.506933
4      40050.178522
           ...     
995    40670.557589
996    33176.622438
997    43471.215897
998    27224.485692
999    28934.414104
Name: Kilométrage, Length: 1000, dtype: float64
    R^2 =  0.6674139307886543
        ==> 66.74% de la variabilité du prix est expliquée par Kilométrage
    On teste lhypothèse HO : beta_1 = 0 au risque alpha = 5%
    p_value :  8.317164806676831e-241
On rejette H0 au risque 5%.
Kilométrage  a bien un impact sur le prix
________________________________________________________________________________________________
Modèle n°  2 permettant de faire une régréssion simple entre le Prix et le/la/l' Année_Fabrication
y_hat =  0      32748.067137
1      31568.610525
2      32394.2

Conclusion : 
La régression est pertinente. 

Les kilométrages, l'année de fabrication et la puissance ont d'après ce test un impact sur le prix de la voiture.

On fait une matrice de corrélation des facteurs afin de vérifier les hypothèses émises précédemment. 

Cette étape permet d'identifier les variables fortement corrélées : 

In [119]:
data_reg.corr()

,Kilométrage,Année_Fabrication,Puissance,Consommation,Nombre_Portes,Vitesse_Max,Marque,Prix
Kilométrage,1.000000,0.003194,0.008297,0.000777,-0.046288,-0.006194,-0.035796,0.816954
Année_Fabrication,0.003194,1.000000,-0.027392,-0.046550,0.012604,0.013367,-0.042143,-0.078235
Puissance,0.008297,-0.027392,1.000000,0.035687,0.025288,-0.013920,-0.016925,0.300363
Consommation,0.000777,-0.046550,0.035687,1.000000,-0.031992,0.011022,-0.039462,0.029347
Nombre_Portes,-0.046288,0.012604,0.025288,-0.031992,1.000000,-0.021306,-0.010635,-0.002437
Vitesse_Max,-0.006194,0.013367,-0.013920,0.011022,-0.021306,1.000000,0.010579,0.019703
Marque,-0.035796,-0.042143,-0.016925,-0.039462,-0.010635,0.010579,1.000000,0.024978
Prix,0.816954,-0.078235,0.300363,0.029347,-0.002437,0.019703,0.024978,1.000000


D'après la matrice de corrélation on constate que le kilométrage est bien impliqué dans le prix de la voiture, il est également le paramètre ayant le plus d'influence. 
En effet, la corrélation entre le prix et le kilométrage est de 0.816954.

La puissance joue également beaucoup, en effet la corrélation entre cette dernière et le prix est de 0.300363 et la p-value est de : 2.6921051334599183e-22, qui fait que l'on rejette H0 au risque 5%. Cette hypothèse peut cependant être mise en doute par la valeur donnée de R² qui est de : 0.09021794012255036 (et semble donc très faible)

Pour l'année de fabrication nous remettons également en doute les résultats obtenus. Sa p-valeur permet de rejetter H0 au risque 5%, cependant, son R² est très faible (0.006120773394306134). De plus en regardant la matrice de corrélation on constate que celle-ci est négative révéllant une relation linéaire inverse entre les deux variables, ou plus communément dis, une anticoréllation. 

A ce stade on regarde la régression linéaire qui est une technique statistique qui vise à modéliser la relation linéaire entre une variable dépendante (ou réponse) et une ou plusieurs variables indépendantes (ou explicatives). La méthode des moindres carrés ordinaires cherche à minimiser la somme des carrés des écarts entre les valeurs observées de la variable dépendante et celles prédites par le modèle.

In [120]:
y =data_reg['Prix']
Colonnes = ['Kilométrage', 'Année_Fabrication','Puissance','Consommation','Nombre_Portes','Vitesse_Max','Marque']

X = data_reg[Colonnes]
     
model = OLS(y,X)
report = model.fit()
report.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                   Prix   R-squared (uncentered):                   0.978
Model:                            OLS   Adj. R-squared (uncentered):              0.977
Method:                 Least Squares   F-statistic:                              6187.
Date:                Tue, 19 Dec 2023   Prob (F-statistic):                        0.00
Time:                        16:34:54   Log-Likelihood:                         -9953.3
No. Observations:                1000   AIC:                                  1.992e+04
Df Residuals:                     993   BIC:                                  1.995e+04
Df Model:                           7                                                  
Covariance Type:            nonrobust                                                  
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Kilométrage           0.0986      0.002     52.469      0.000       0.095       0.102
Année_Fabrication     1.0691      0.650      1.646      0.100      -0.206       2.344
Puissance            47.6474      2.523     18.884      0.000      42.696      52.599
Consommation          8.2549      5.918      1.395      0.163      -3.358      19.868
Nombre_Portes       274.6102    139.567      1.968      0.049       0.730     548.490
Vitesse_Max           7.2057      3.756      1.919      0.055      -0.164      14.576
Marque              776.2738    199.236      3.896      0.000     385.302    1167.246
==============================================================================
Omnibus:                        1.449   Durbin-Watson:                   2.112
Prob(Omnibus):                  0.485   Jarque-Bera (JB):                1.350
Skew:                          -0.086   Prob(JB):                        0.509
Kurtosis:                       3.051   Cond. No.                     2.10e+05
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[3] The condition number is large, 2.1e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

R-squared (R²) : Notre modèle a une valeur de 0.978. Cela indique que le modèle explique environ 97.8% de la variabilité dans le prix de voitures. 

F-statistic : La valeur pour ce modèle est de: 0.00 avec une probabilité très proche de zéro, indiquant que le modèle global est statistiquement significatif.

Les coefficients associés à chaque variable indépendante révèle pour la variable Kilométrage, un coefficient de 0.0986. Cela révèle une augmentation d'une unité dans le Kilométrage est associée à une augmentation de 0.0986 unité dans le prix. 

P-value : cette valeur indique la probabilité d'observer une valeur du coefficient aussi extrême que celle observée, si l'hypothèse nulle selon laquelle le coefficient est nul est vraie. Des p-values faibles (p < 0.05) suggèrent que le coefficient est statistiquement significatif. Ainsi les p-value sont d'après ce test significatives pour l'année de fabrication, la consommation et la vitesse maximale. Cela change des résultats obtenus précédemment. 


## Conclusion pour la partie Régréssion du prix des voitures : 
Le modèle semble expliquer une grande partie de la variabilité observé dans le prix des véhicules. 
On constate cependant que toutes les valeurs de régréssion calculé ne montrent pas la même chose. 
Il est ainsi nécéssaire de regarder comment se comporte le prix selon d'autres techniques d’analyse de données et de machine learning :